# Генерация ответов по ранее сгенерированным вопросам

In [1]:
from source.tinydb_manager import TinyDB_manager
from source.chroma_manager import ChromaDBManager
from source.llm_manager import LMStudioClient
from prompt.prompts import QA_context
import json
import time
from typing import Dict, Any, List
from tqdm import tqdm
import logging

C:\Users\Igorexy\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Config

In [2]:
# LANG = '_RUS'
LANG = ''   # English

In [3]:
# Путь к файлу с тестами необходимой модели эмбеддингов
# RAG_result_path = 'benchmark/input_LLM/multi-qa-MiniLM-L6-cos-v1-embedding.json'
# chroma_config_path = "config/embedding/multi-qa-MiniLM-L6-cos-v1.yaml"
chroma_config_path = "config/embedding/questions_gen.yaml"
# chroma_config_path = "config/embedding/questions_rus.yaml"

# LLM_config_path = "config/LLM/qwen2.5_7b_instruct.yaml"
# LLM_config_path = "config/LLM/meta-llama-3.1-8b-instruct.yaml"
# LLM_config_path = "config/LLM/gpt4chan-8b.yaml"
# LLM_config_path = "config/LLM/mistral-7b-instruct-v0.3.yaml"
# LLM_config_path = "config/LLM/qwen2.5-7b-instruct-1m.yaml"
# LLM_config_path = "config/LLM/llama-3.2-3b-instruct.yaml"
# LLM_config_path = "config/LLM/qwen3-1.7b.yaml"
# LLM_config_path = "config/LLM/gemma-3-1b-it.yaml"
# LLM_config_path = "config/LLM/deepseek-r1-distill-qwen-1.5b.yaml"
# LLM_config_path = "config/LLM/phi-3.1-mini-128k-instruct@q4_k_m.yaml"
# LLM_config_path = "config/LLM/phi-3.1-mini-128k-instruct@iq3_m.yaml"
LLM_config_path = "config/LLM/deepseek-r1-0528-qwen3-8b.yaml"

LLM_config = LMStudioClient.load_config(LLM_config_path)

LLM_NAME = LLM_config['model_name']

result_path = f"benchmark/output_LLM{LANG}/{LLM_NAME}.json".replace('-', '_')

chroma_manager = ChromaDBManager(chroma_config_path)
tiny_manager = TinyDB_manager(result_path)

llm_manager = LMStudioClient(LLM_config_path)

# Считывание JSON-файла в словарь
# with open(RAG_result_path, 'r', encoding='utf-8') as f:
#     RAG_data = json.load(f)

e:\ImportantFiles\Documents\University\Magic App\source\llm_manager.py:9: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  self.llm = ChatOpenAI(


# Выбираем часть датасета

In [5]:
# import random

# len(RAG_data.keys())
# random_50 = random.sample(list(RAG_data.keys()), 50)

In [4]:
selected_keys = ['0105522v1',
 '2309.00201v2',
 '2409.08175v1',
 '2002.04991v1',
 '2402.06452v1',
 '0504056v1',
 '2010.14616v1',
 '1505.02965v2',
 '2405.03720v1',
 '2402.04453v1',
 '1104.4174v1',
 '2108.10078v1',
 '2306.13586v1',
 '2303.02715v1',
 '2111.08438v1',
 '2501.12927v1',
 '2102.02850v3',
 '2012.02526v2',
 '2110.01856v1',
 '2009.07708v1',
 '1511.05520v1',
 '2005.11394v1',
 '2106.08671v2',
 '1805.04825v1',
 '2502.07081v2',
 '2501.16247v1',
 '2302.02016v1',
 '1211.1127v1',
 '2501.06863v1',
 '1804.03313v1',
 '2004.02401v1',
 '2210.00770v1',
 '2305.11304v2',
 '2211.00176v1',
 '2305.10344v2',
 '2007.04446v1',
 '2008.09306v1',
 '2211.15869v1',
 '2006.02158v2',
 '1702.06794v1',
 '2209.15308v1',
 '1911.05640v2',
 '2206.06828v1',
 '2010.06026v1',
 '2007.04911v2',
 '2208.10651v1',
 '1205.1938v1',
 '2307.12343v1',
 '2405.09305v1',
 '2401.17200v1',
 '2312.06833v2',
 '2412.06481v1',
 '2108.11053v1',
 '2405.06323v1',
 '2005.13449v1',
 '2206.03950v3',
 '2311.10905v1',
 '1706.01109v2',
 '1808.05443v1',
 '1507.06923v1',
 '2203.14474v1',
 '1307.6275v1',
 '0910.4472v2',
 '2209.05559v6',
 '1811.07311v2',
 '2304.02781v1',
 '2212.08930v4',
 '1312.7118v1',
 '1402.5724v1',
 '2103.15593v1',
 '1805.00980v1',
 '1112.6281v1',
 '1806.01756v1',
 '2012.08044v2',
 '2211.08282v1',
 '2202.01214v1',
 '1701.00867v1',
 '2501.14940v3',
 '1301.3583v4',
 '1303.2104v1',
 '0909.2332v1',
 '1711.01739v1',
 '1808.10009v1',
 '2308.14478v1',
 '2110.04942v1',
 '1510.05684v2',
 '2306.08001v1',
 '2010.05522v1',
 '2010.08034v1',
 '2401.06793v1',
 '1506.01055v1',
 '1612.07993v1',
 '2106.12798v2',
 '2305.03360v1',
 '1212.0238v1',
 '0010231v1',
 '1104.4188v1',
 '2108.01468v1',
 '2101.06480v1',
 '2106.03259v1',
 '2006.11014v1',
 '2305.06063v2',
 '2410.12598v2',
 '2204.00778v1',
 '1607.07625v1',
 '2003.09873v1',
 '2304.12729v2',
 '1811.07579v2',
 '1408.3002v1',
 '1803.07418v1',
 '1412.2352v1',
 '2401.00974v1',
 '1904.10551v1',
 '2406.16106v1',
 '1702.07956v5',
 '1805.00559v1',
 '2004.07136v1',
 '2007.06559v2',
 '1805.02716v1',
 '2006.13307v1',
 '0012536v1',
 '1302.4207v1',
 '1910.12249v1',
 '1502.06254v2',
 '1710.06798v1',
 '2308.02870v1',
 '2005.03476v2',
 '2002.07971v2',
 '2210.08288v1',
 '2107.02331v1',
 '1910.12207v1',
 '1807.04739v1',
 '2003.08445v1',
 '1902.01544v1',
 '2106.13743v1',
 '1810.11363v1',
 '1805.11474v3',
 '2104.04375v1',
 '2409.18583v1',
 '2410.20161v1',
 '1705.04153v1',
 '2406.03348v1',
 '1908.06951v1',
 '1609.01228v1',
 '2111.15258v1',
 '2411.01881v2',
 '1503.03168v1',
 '2309.00626v1',
 '1711.11008v1',
 '2304.02260v1',
 '1812.02897v3',
 '2012.03575v1',
 '2204.12244v1',
 '2502.08869v1',
 '1806.08874v1',
 '2005.09148v1',
 '2308.08682v1',
 '1703.02910v1',
 '2204.09021v1',
 '1902.07068v1',
 '2006.10562v4',
 '2008.01171v1',
 '1706.09796v3',
 '2001.09608v1',
 '2007.05035v1',
 '1104.4193v1',
 '0804.0099v1',
 '2408.09908v2',
 '1705.03921v1',
 '0501077v1',
 '1808.08111v1',
 '2308.11924v1',
 '1704.00405v2',
 '2006.04059v1',
 '2410.18950v1',
 '2007.02259v1',
 '1105.0522v1',
 '1902.06127v2',
 '1908.11215v1',
 '2005.00524v1',
 '1809.06995v1',
 '2309.06659v2',
 '2104.13386v1',
 '2304.13812v1',
 '2109.13233v1',
 '1706.07525v1',
 '2501.16374v2',
 '2305.10114v1',
 '1704.01664v1',
 '1710.08952v1',
 '1603.03719v1',
 '2210.11024v1',
 '2207.02763v1',
 '2101.05840v1',
 '2408.15452v1',
 '2302.02756v1',
 '2208.11873v1',
 '1709.05067v1',
 '2009.01564v2',
 '2209.03032v1']

In [5]:
# filtered_dict = {key: RAG_data[key] for key in selected_keys if key in RAG_data}

# print(filtered_dict)

# Получаем контекст

## Прошлая версия

#### функция полноценного теста с 2 вопросами

In [5]:
def process_and_generate_answers_2_questions(
    full_data: Dict[str, Any],
    existing_answer: List[str],
    chroma_manager,
    llm_manager,
    tiny_manager
):
    """full_data: Dict[str, Any], - Словарь с вопросами
    existing_answer: List[str],
    chroma_manager,
    llm_manager,
    tiny_manager"""
    all_chroma_ids = set(chroma_manager.get_all_ids())

    for doc_id, questions in tqdm(full_data.items()):
        if doc_id in existing_answer:
            continue  # пропускаем, если уже есть ответ

        result_record = {"id": doc_id, doc_id: {}}  # итоговая структура для tiny

        for question_key, question_data in questions.items():
            search_groups = question_data.get("search_ids", [])
            if not search_groups:
                continue

            # собираем все id из search_ids (обычно одна группа)
            flat_ids = list({item for sublist in search_groups for item in sublist})

            # фильтрация на всякий случай (если в Chroma нет документа)
            filtered_ids = [sid for sid in flat_ids if sid in all_chroma_ids]

            # достаем документы из Chroma
            docs_by_id = chroma_manager.get_documents_by_ids(filtered_ids)

            # объединяем документы в единый контекст
            context = "\n\n".join(doc_info["document"] for doc_info in docs_by_id.values())

            # используем только первый найденный документ
            # context = ""
            # if filtered_ids:
            #     first_id = filtered_ids[0]
            #     doc_info = chroma_manager.get_documents_by_ids([first_id])
            #     if doc_info and first_id in doc_info:
            #         context = doc_info[first_id]["document"]

            # Извлекаем вопрос
            user_question = question_data.get("question", '')

            # формируем prompt 
            prompt = QA_context.SYSTEM_QA_SHORT.format(question=user_question, context=context)

            # замер времени генерации
            start_time = time.time()
            llm_answer = llm_manager.post_completion(prompt)
            end_time = time.time()

            generation_time = round(end_time - start_time, 3)

            # обновляем вопрос
            enriched_question = question_data.copy()
            enriched_question["context"] = context
            enriched_question["answer_llm"] = llm_answer
            enriched_question["generation_time"] = generation_time

            result_record[doc_id][question_key] = enriched_question
            break
        # сохраняем результат
        tiny_manager.save_result(result_record)


#### Функция теста с 1 вопросом

In [ ]:
# Логирование в файл
logging.basicConfig(
    filename='log/llm_test.log',
    filemode='a',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

logger = logging.getLogger(__name__)

def process_and_generate_answers(data: Dict[str, Any], 
                      existing_answer, 
                      chroma_manager, 
                      llm_manager, 
                      tiny_manager):
    try:
        all_ids = set(chroma_manager.get_all_ids())
    except Exception as e:
        logger.error(f"Ошибка при получении всех ID из Chroma: {e}")
        return

    logger.info(f"Всего документов в базе: {len(data)}")

    for doc_id in tqdm(data.keys(), desc="Обработка документов"):
        if doc_id in existing_answer:
            continue

        questions = data.get(doc_id, {})
        question_data = questions.get("question_1")
        if not question_data:
            logger.warning(f"В документе {doc_id} отсутствует question_1.")
            continue

        question_text = question_data["question"]
        search_ids_nested = question_data.get("search_ids", [])

        # Извлекаем только первый ID из первого списка
        if not search_ids_nested or not search_ids_nested[0]:
            logger.warning(f"В {doc_id} пустой search_ids.")
            continue

        first_related_id = search_ids_nested[0][0]

        if first_related_id not in all_ids:
            logger.warning(f"Документ {first_related_id} из search_ids не найден в Chroma.")
            continue

        try:
            document_entry = chroma_manager.get_documents_by_ids([first_related_id])
            document_text = document_entry.get(first_related_id, {}).get("document", "")
        except Exception as e:
            logger.error(f"Ошибка при получении документа {first_related_id} для {doc_id}: {e}")
            continue

        if not document_text:
            logger.warning(f"Пустой документ для {first_related_id}")
            continue

        # Формируем промпт
        try:
            prompt = QA_context.SYSTEM_QA_SHORT.format(
                question=question_text,
                context=document_text
            )
        except Exception as e:
            logger.error(f"Ошибка при форматировании промпта для {doc_id}: {e}")
            continue

        try:
            start_time = time.time()
            answer_llm = llm_manager.post_completion(prompt)
            generation_time = time.time() - start_time
        except Exception as e:
            logger.error(f"Ошибка при генерации ответа LLM для {doc_id}: {e}")
            continue

        result_record = {
            "id": doc_id,
            "question_1": {
                **question_data,
                "answer_llm": answer_llm,
                "generation_time": generation_time
            }
        }

        try:
            tiny_manager.save_result(record=result_record)
            logger.info(f"Успешно сохранён результат для {doc_id}")
        except Exception as e:
            logger.error(f"Ошибка при сохранении результата для {doc_id}: {e}")

## функция теста с 1 вопросом без RAG поиска  
Последняя актуальная версия для тестов

In [5]:
import time
import logging
from tqdm import tqdm

# Настройка логирования
logging.basicConfig(
    filename='log/llm_generation.log',
    filemode='a',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

logger = logging.getLogger(__name__)

def process_and_generate_answers_latest(question_ids_list, 
                                 existing_answer, 
                                 chroma_manager, 
                                 llm_manager, 
                                 tiny_manager):
    """Если question_ids_list = 'all', то все документы, иначе переменная ожидает список ids"""
    try:
        if question_ids_list == 'all':
            question_ids_list = chroma_manager.get_all_ids()
        all_docs = chroma_manager.get_documents_by_ids(question_ids_list)
    except Exception as e:
        logger.error(f"Ошибка при извлечении документов из Chroma: {e}")
        return

    for doc_id in tqdm(question_ids_list, desc="Генерация ответов"):
        if doc_id in existing_answer:
            logger.info(f"Пропущен {doc_id} — уже существует.")
            continue

        doc_entry = all_docs.get(doc_id)
        if not doc_entry:
            logger.warning(f"Документ {doc_id} не найден в Chroma.")
            continue

        metadata = doc_entry.get("metadata", {})
        document_text = doc_entry.get("document", "")
        
        question_data = eval(metadata.get("questions"))
        question_1 = question_data.get("question_1")
        answer_1 = question_data.get("answer_1")
        if not question_1:
            logger.warning(f"В метаданных документа {doc_id} нет question_1.")
            continue

        question_text = question_1
        answer_text = answer_1
        if not question_text:
            logger.warning(f"Вопрос отсутствует в question_1 для {doc_id}.")
            continue

        if not document_text:
            logger.warning(f"Пустой текст документа для {doc_id}.")
            continue

        # Формируем промпт
        try:
            if LANG == '_RUS':
                prompt = QA_context.SYSTEM_QA_SHORT_RUS.format(
                    question=question_text,
                    context=document_text
                )
            else:
                prompt = QA_context.SYSTEM_QA_SHORT.format(
                    question=question_text,
                    context=document_text
                )
        except Exception as e:
            logger.error(f"Ошибка при создании промпта для {doc_id}: {e}")
            continue

        # Генерация ответа и замер времени
        try:
            start_time = time.time()
            answer_llm = llm_manager.post_completion(prompt)
            generation_time = time.time() - start_time
        except Exception as e:
            logger.error(f"Ошибка генерации LLM-ответа для {doc_id}: {e}")
            continue

        result_record = {
            "id": doc_id,
            "question_1": {
                "question": question_text,
                "answer": answer_text,
                "answer_llm": answer_llm,
                "generation_time": generation_time
            }
        }

        try:
            tiny_manager.save_result(record=result_record)
            logger.info(f"Успешно сохранён результат для \n{result_record}")
        except Exception as e:
            logger.error(f"Ошибка при сохранении результата для {doc_id}: {e}")


### Проверка наличия вопросов

In [6]:
existing_answer = []

if tiny_manager.count() > 0:
    existing_answer = tiny_manager.get_all_ids()
len(existing_answer)

19

In [ ]:
process_and_generate_answers_latest(#question_ids_list='all', 
                                    question_ids_list=selected_keys, 
                             existing_answer=existing_answer,
                             chroma_manager=chroma_manager,
                             llm_manager=llm_manager,
                             tiny_manager=tiny_manager
                             )

Генерация ответов:  10%|▉         | 19/200 [08:02<2:03:06, 40.81s/it]